In [0]:
silver_df = spark.table("workspace.default.silver_telco_churn")

In [0]:
silver_df.count()
silver_df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [0]:
from pyspark.sql.functions import when, col

features_df = (
    silver_df
    .withColumn(
        "SeniorCitizen",
        when(col("SeniorCitizen") == "Yes", 1)
        .when(col("SeniorCitizen") == "No", 0)
        .otherwise(None)
    )
    .select(
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "tenure",
        "PhoneService",
        "MultipleLines",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "Contract",
        "PaperlessBilling",
        "PaymentMethod",
        "MonthlyCharges",
        "TotalCharges",
        "Churn"
    )
)


In [0]:
features_df.write.mode("overwrite").saveAsTable(
    "workspace.default.gold_telco_churn_features"
)

In [0]:
df = spark.table("workspace.default.gold_telco_churn_features")

In [0]:
df.printSchema()
df.select("Churn").groupBy("Churn").count().show()

root
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5174|
|  Yes| 1869|
+-----+-----+



In [0]:
from pyspark.sql.functions import when

df = df.withColumn(
    "label",
    when(df.Churn == "Yes", 1).otherwise(0)
)

In [0]:
categorical_cols = [
    "gender", "Partner", "Dependents", "PhoneService",
    "MultipleLines", "InternetService", "OnlineSecurity",
    "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies", "Contract",
    "PaperlessBilling", "PaymentMethod"
]

numerical_cols = [
    "SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"
]

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep")
    for col in categorical_cols
]

encoders = [
    OneHotEncoder(inputCol=f"{col}_idx", outputCol=f"{col}_ohe")
    for col in categorical_cols
]


In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[f"{col}_ohe" for col in categorical_cols] + numerical_cols,
    outputCol="features"
)

In [0]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

In [0]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol="features", labelCol="label")


In [0]:
from pyspark.ml import Pipeline

pipeline = Pipeline(
    stages=indexers + encoders + [assembler, lr]
)

In [0]:
from pyspark.sql.functions import col, sum

df.select(
    sum(col("SeniorCitizen").isNull().cast("int")).alias("SeniorCitizen_nulls"),
    sum(col("tenure").isNull().cast("int")).alias("tenure_nulls"),
    sum(col("MonthlyCharges").isNull().cast("int")).alias("MonthlyCharges_nulls"),
    sum(col("TotalCharges").isNull().cast("int")).alias("TotalCharges_nulls")
).show()

+-------------------+------------+--------------------+------------------+
|SeniorCitizen_nulls|tenure_nulls|MonthlyCharges_nulls|TotalCharges_nulls|
+-------------------+------------+--------------------+------------------+
|                  0|           0|                   0|                11|
+-------------------+------------+--------------------+------------------+



In [0]:
df = df.fillna({
    "TotalCharges": 0.0,
    "MonthlyCharges": 0.0,
    "tenure": 0,
    "SeniorCitizen": 0
})

In [0]:
df.select(
    sum(col("TotalCharges").isNull().cast("int")).alias("TotalCharges_nulls")
).show()

+------------------+
|TotalCharges_nulls|
+------------------+
|                 0|
+------------------+



In [0]:
model = pipeline.fit(train_df)

---------------------------------------------------------------------------
SparkException                            Traceback (most recent call last)
File <command-6999136843369320>, line 1
----> 1 model = pipeline.fit(train_df)

File /databricks/python_shell/lib/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/python/lib/python3.12/site-packages/pyspark/ml/base.py:203, in Estimator.fit(self, dataset, params)
    201         return self.copy(params)._fit(dataset)
    202     else:
--> 203         return self._fit(dataset)
    204 else:
    205     raise TypeError(
    206         "Params must be either a param map or a list/tuple of param maps, "
    207         "but got %s." % type(params)
    208     )

File /databricks/python/